### **Modelos secuencia a secuencia**

Un modelo secuencia a secuencia (Seq2Seq) es una arquitectura de red neuronal diseñada para transformar una secuencia de entrada en una secuencia de salida, y es especialmente útil para tareas donde la longitud de las secuencias de entrada y salida puede diferir. 

Los modelos Seq2Seq son comúnmente utilizados en aplicaciones como traducción automática, resumen de texto, y generación de texto.

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# S: Símbolo que marca el inicio de la entrada de decodificación
# E: Símbolo que marca el final de la salida de decodificación
# P: Símbolo de relleno para completar secuencias cortas

# Parámetros del modelo
n_step = 5          # Longitud de las secuencias (pasos de tiempo)
n_hidden = 128      # Tamaño del estado oculto de las RNNs

# Definición del conjunto de caracteres y su mapeo numérico
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}

# Datos de entrenamiento (pares de palabras)
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

n_class = len(num_dic)   # Número de clases = número de caracteres posibles
batch_size = len(seq_data)  # Número de pares de datos

# Función para preparar el lote de entrenamiento
def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))  # Rellenar con 'P' si es necesario

        input_seq = [num_dic[n] for n in seq[0]]              # Convertir caracteres a índices
        output_seq = [num_dic[n] for n in ('S' + seq[1])]      # Agregar símbolo 'S' al inicio de la salida
        target = [num_dic[n] for n in (seq[1] + 'E')]          # Agregar símbolo 'E' al final como target

        input_batch.append(np.eye(n_class)[input_seq])         # One-hot encoding de entrada
        output_batch.append(np.eye(n_class)[output_seq])       # One-hot encoding de salida
        target_batch.append(target)                            # Target como índices (no one-hot)

    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Función para preparar un lote de prueba dado un input_word
def make_testbatch(input_word):
    input_w = input_word + 'P' * (n_step - len(input_word))  # Rellenar si es necesario
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]     # Salida inicializada con 'S' + relleno

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Definición del modelo Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        # Capa RNN para codificador
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # Capa RNN para decodificador
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # Capa totalmente conectada para predecir clases
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        # Cambiar dimensiones: [n_step, batch_size, n_class]
        enc_input = enc_input.transpose(0, 1)
        dec_input = dec_input.transpose(0, 1)

        # Codificar
        _, enc_states = self.enc_cell(enc_input, enc_hidden)

        # Decodificar
        outputs, _ = self.dec_cell(dec_input, enc_states)

        # Clasificación de la salida
        modelo = self.fc(outputs)  # [n_step+1, batch_size, n_class]
        return modelo

# Inicializar modelo, función de pérdida y optimizador
modelo = Seq2Seq()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelo.parameters(), lr=0.001)

# Preparar los lotes de entrada/salida/target
input_batch, output_batch, target_batch = make_batch()

# Entrenamiento del modelo
for epoch in range(5000):
    # Inicializar el estado oculto
    hidden = torch.zeros(2, batch_size, n_hidden)

    optimizer.zero_grad()

    # Propagación hacia adelante
    output = modelo(input_batch, hidden, output_batch)
    output = output.transpose(0, 1)  # [batch_size, n_step+1, n_class]

    loss = 0
    for i in range(batch_size):
        # Calcular la pérdida para cada muestra
        loss += criterion(output[i], target_batch[i])

    if (epoch + 1) % 1000 == 0:
        print(f'Época: {epoch + 1:04d}, costo = {loss.item():.6f}')

    loss.backward()
    optimizer.step()

# Función para traducir una palabra nueva
def translate(word):
    # Preparar lotes
    input_batch, output_batch = make_testbatch(word)
    hidden = torch.zeros(2, 1, n_hidden)

    # Propagación hacia adelante
    output = modelo(input_batch, hidden, output_batch)
    
    # Obtener predicciones: tomar la clase con mayor probabilidad
    predict = output.data.max(2, keepdim=True)[1]
    decoded = [char_arr[i.item()] for i in predict.squeeze()]

    # Detenerse en 'E' si aparece
    if 'E' in decoded:
        end = decoded.index('E')
        translated = ''.join(decoded[:end])
    else:
        translated = ''.join(decoded)

    return translated.replace('P', '')  # Eliminar los rellenos

# Pruebas de traducción
print('test')
print('man ->', translate('man'))
print('mans ->', translate('mans'))
print('king ->', translate('king'))
print('black ->', translate('black'))
print('upp ->', translate('upp'))


### **Greedy, beam search y selección aleatoria con temperatura**

Podemos implementar las estrategias de decodificación greedy, beam search y selección aleatoria con temperatura, podemos modificar el código dado agregando funciones específicas para cada método de decodificación.

**Estrategia greedy**

La estrategia greedy selecciona el token con la mayor probabilidad en cada paso de decodificación.


In [ ]:
import numpy as np
import torch
import torch.nn as nn

# Parámetros principales
n_step = 5          # Número de pasos de tiempo (longitud máxima de secuencias)
n_hidden = 128      # Número de neuronas ocultas en las RNNs

# Definición del conjunto de caracteres y mapeo de caracteres a índices
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}

# Datos de entrenamiento: pares de secuencias
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

n_class = len(num_dic)     # Número de clases = número total de caracteres
batch_size = len(seq_data) # Tamaño del lote = número de pares

# Función para preparar el lote de entrenamiento
def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            # Rellenar las secuencias con 'P' si son más cortas que n_step
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        # Convertir caracteres a índices
        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])] # Agrega símbolo 'S' al inicio
        target = [num_dic[n] for n in (seq[1] + 'E')]      # Agrega símbolo 'E' al final

        # Codificar en one-hot
        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target) # El target es una secuencia de índices (no one-hot)

    # Convertir listas a tensores de PyTorch
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Función para preparar el lote de prueba para una palabra de entrada
def make_testbatch(input_word):
    input_w = input_word + 'P' * (n_step - len(input_word)) # Rellenar si es necesario
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]   # Salida empieza con 'S' + relleno

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    # Convertir a tensores y agregar dimensión batch
    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Definición de la arquitectura del modelo Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        # Codificador RNN de 2 capas con dropout
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # Decodificador RNN de 2 capas con dropout
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # Capa completamente conectada para predecir caracteres
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        # Transponer para que el tiempo esté en la primera dimensión
        enc_input = enc_input.transpose(0, 1) # [n_step, batch_size, n_class]
        dec_input = dec_input.transpose(0, 1) # [n_step, batch_size, n_class]

        # Propagación codificadora
        _, enc_states = self.enc_cell(enc_input, enc_hidden)

        # Propagación decodificadora
        outputs, _ = self.dec_cell(dec_input, enc_states)

        # Aplicar capa final para obtener distribución de clases
        modelo = self.fc(outputs)

        return modelo, enc_states

# Inicialización del modelo, función de pérdida y optimizador
modelo = Seq2Seq()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelo.parameters(), lr=0.001)

# Preparar los lotes de entrada, salida y objetivos
input_batch, output_batch, target_batch = make_batch()

# Entrenamiento del modelo
for epoch in range(5000):
    modelo.train() # Modo entrenamiento
    hidden = torch.zeros(2, batch_size, n_hidden) # Estado oculto inicial

    optimizer.zero_grad() # Reiniciar gradientes
    output, _ = modelo(input_batch, hidden, output_batch)
    output = output.transpose(0, 1) # [batch_size, n_step, n_class]

    # Calcular pérdida sumando las pérdidas por elemento
    loss = 0
    for i in range(batch_size):
        loss += criterion(output[i], target_batch[i])

    if (epoch + 1) % 1000 == 0:
        print(f'Época: {epoch + 1:04d}, costo = {loss.item():.6f}')

    loss.backward() # Retropropagación
    optimizer.step() # Actualización de parámetros

# Función para traducir una nueva palabra usando decodificación greedy
def translate(word):
    modelo.eval() # Modo evaluación
    with torch.no_grad(): # No calcular gradientes
        input_w = word + 'P' * (n_step - len(word)) # Rellenar si es necesario
        input_seq = [num_dic[n] for n in input_w]
        input_batch = np.eye(n_class)[input_seq]
        input_batch = torch.FloatTensor(input_batch).unsqueeze(0) # [1, n_step, n_class]

        hidden = torch.zeros(2, 1, n_hidden) # Estado oculto inicial para prueba
        enc_input = input_batch.transpose(0, 1)

        # Codificar la entrada
        _, enc_states = modelo.enc_cell(enc_input, hidden)

        # Inicializar entrada del decodificador con símbolo 'S'
        decoder_input = torch.zeros(1, 1, n_class)
        decoder_input[0, 0, num_dic['S']] = 1.0

        decoded = []
        for _ in range(n_step + 1):
            # Paso de decodificación
            output, enc_states = modelo.dec_cell(decoder_input, enc_states)
            output = modelo.fc(output.squeeze(0))  # [1, n_class]

            _, topi = output.topk(1) # Elegir el índice de mayor probabilidad
            next_token = topi.item()
            char = char_arr[next_token]

            if char == 'E': # Finalizar si se predice 'E'
                break
            if char != 'P': # Ignorar símbolos de relleno
                decoded.append(char)

            # Actualizar entrada del decodificador
            decoder_input = torch.zeros(1, 1, n_class)
            decoder_input[0, 0, next_token] = 1.0

        translated = ''.join(decoded)
        return translated

# Pruebas de traducción
print('Prueba de decodificación greedy:')
print('man ->', translate('man'))
print('mans ->', translate('mans'))
print('king ->', translate('king'))
print('black ->', translate('black'))
print('upp ->', translate('upp'))


**Estrategia beam search**

Beam Search mantiene las mejores k secuencias en cada paso, lo que permite explorar múltiples caminos en la decodificación y seleccionar la secuencia más probable.

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# Parámetros principales
n_step = 5          # Número de pasos de tiempo (longitud fija de secuencias)
n_hidden = 128      # Tamaño del estado oculto en las RNNs

# Definición de caracteres y mapeo a índices numéricos
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}

# Datos de entrenamiento: pares de palabras
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

n_class = len(num_dic)     # Número total de caracteres (clases)
batch_size = len(seq_data) # Número de pares de datos

# Función para preparar el lote de entrenamiento
def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))  # Rellenar con 'P' si la secuencia es más corta que n_step

        input_seq = [num_dic[n] for n in seq[0]]              # Indices para la entrada
        output_seq = [num_dic[n] for n in ('S' + seq[1])]      # 'S' como inicio de salida
        target = [num_dic[n] for n in (seq[1] + 'E')]          # 'E' como final para el target

        input_batch.append(np.eye(n_class)[input_seq])         # One-hot encoding de entrada
        output_batch.append(np.eye(n_class)[output_seq])       # One-hot encoding de salida
        target_batch.append(target)                            # Target como índices

    # Convertir las listas a arrays de numpy y luego a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Función para preparar un lote de prueba (para inferencia)
def make_testbatch(input_word):
    input_w = input_word + 'P' * (n_step - len(input_word))  # Rellenar si es necesario
    input_seq = [num_dic.get(n, num_dic['P']) for n in input_w]  # Convertir a índices
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]    # Salida comienza con 'S' y 'P's

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Definición del modelo Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)  # Codificador RNN
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)  # Decodificador RNN
        self.fc = nn.Linear(n_hidden, n_class)  # Capa completamente conectada para predecir caracteres

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1)  # [n_step, batch_size, n_class]
        dec_input = dec_input.transpose(0, 1)  # [n_step, batch_size, n_class]

        _, enc_states = self.enc_cell(enc_input, enc_hidden)   # Codificación
        outputs, _ = self.dec_cell(dec_input, enc_states)      # Decodificación

        modelo = self.fc(outputs)  # Salida final (predicción de caracteres)
        return modelo, enc_states

# Inicializar modelo, función de pérdida y optimizador
modelo = Seq2Seq()
criterion = nn.CrossEntropyLoss()                     # Función de pérdida de entropía cruzada
optimizer = torch.optim.Adam(modelo.parameters(), lr=0.001)  # Optimizador Adam

# Preparar los lotes de entrenamiento
input_batch, output_batch, target_batch = make_batch()

# Entrenamiento del modelo
for epoch in range(5000):
    modelo.train()                                    # Modo entrenamiento
    hidden = torch.zeros(2, batch_size, n_hidden)     # Inicializar estado oculto

    optimizer.zero_grad()                             # Resetear gradientes
    output, _ = modelo(input_batch, hidden, output_batch)  # Forward pass
    output = output.transpose(0, 1)                   # [batch_size, n_step, n_class]

    # Calcular pérdida acumulada para el batch
    loss = 0
    for i in range(batch_size):
        loss += criterion(output[i], target_batch[i])

    if (epoch + 1) % 1000 == 0:                        # Imprimir cada 1000 épocas
        print(f'Época: {epoch + 1:04d}, costo = {loss.item():.6f}')

    loss.backward()                                   # Backpropagation
    optimizer.step()                                  # Actualizar parámetros

# Función de decodificación usando beam search
def translate_beam_search(word, beam_size=3, max_dec_steps=10):
    modelo.eval()  # Cambiar a modo evaluación
    with torch.no_grad():  # No calcular gradientes
        input_w = word + 'P' * (n_step - len(word))  # Rellenar si necesario
        input_seq = [num_dic.get(n, num_dic['P']) for n in input_w]
        input_batch = np.eye(n_class)[input_seq]
        input_batch = torch.FloatTensor(input_batch).unsqueeze(0)

        hidden = torch.zeros(2, 1, n_hidden)          # Inicializar estado oculto
        enc_input = input_batch.transpose(0, 1)

        _, enc_states = modelo.enc_cell(enc_input, hidden)  # Codificación

        # Inicializar haz de búsqueda con el símbolo 'S'
        beam = [{
            'sequence': [num_dic['S']],
            'hidden': enc_states.clone(),
            'score': 0.0
        }]
        completed_sequences = []

        for _ in range(max_dec_steps):
            new_beam = []
            for seq in beam:
                last_token = seq['sequence'][-1]

                if last_token == num_dic['E']:  # Si se alcanza el final
                    completed_sequences.append(seq)
                    continue

                # Crear entrada para decodificador
                decoder_input = torch.zeros(1, 1, n_class)
                decoder_input[0, 0, last_token] = 1.0

                dec_output, dec_hidden = modelo.dec_cell(decoder_input, seq['hidden'])
                logits = modelo.fc(dec_output.squeeze(0))
                log_probs = nn.functional.log_softmax(logits, dim=1)

                # Tomar top-k tokens más probables
                topk_log_probs, topk_indices = torch.topk(log_probs, beam_size, dim=1)

                # Crear nuevas secuencias candidatas
                for i in range(beam_size):
                    token = topk_indices[0, i].item()
                    score = seq['score'] + topk_log_probs[0, i].item()
                    new_seq = seq['sequence'] + [token]
                    new_hidden = dec_hidden.clone()
                    new_beam.append({
                        'sequence': new_seq,
                        'hidden': new_hidden,
                        'score': score
                    })

            # Mantener solo las mejores beam_size secuencias
            new_beam = sorted(new_beam, key=lambda x: x['score'], reverse=True)
            beam = new_beam[:beam_size]

            # Terminar si todas las secuencias han llegado a 'E'
            if len(completed_sequences) >= beam_size:
                break

        if not completed_sequences:  # Si no hay completas, usar actuales
            completed_sequences = beam

        completed_sequences = sorted(completed_sequences, key=lambda x: x['score'], reverse=True)
        best_sequence = completed_sequences[0]['sequence']

        # Decodificar la mejor secuencia
        decoded = [char_arr[i] for i in best_sequence]

        if 'E' in decoded:
            end = decoded.index('E')
            decoded = decoded[:end]

        translated = ''.join([char for char in decoded if char not in ['S', 'P']])
        return translated

# Prueba de la decodificación beam search
print('\nPrueba de decodificación beam search:')
print('man ->', translate_beam_search('man'))
print('mans ->', translate_beam_search('mans'))
print('king ->', translate_beam_search('king'))
print('black ->', translate_beam_search('black'))
print('upp ->', translate_beam_search('upp'))


**Selección aleatoria con temperatura**

La selección aleatoria con temperatura ajusta las probabilidades de los tokens antes de muestrear de la distribución, permitiendo más exploración en la generación.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Parámetros principales
n_step = 5          # Número de pasos de tiempo (longitud fija de la secuencia)
n_hidden = 128      # Tamaño del estado oculto de las RNNs

# Definición del conjunto de caracteres y su mapeo a índices numéricos
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}

# Datos de entrenamiento: pares de palabras
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

n_class = len(num_dic)      # Número total de caracteres/clases
batch_size = len(seq_data)  # Número de pares de datos en el lote

# Función para crear el lote de entrenamiento
def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            # Rellenar las palabras con 'P' hasta alcanzar n_step caracteres
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        # Convertir caracteres a índices
        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])]  # Añadir 'S' al inicio de la secuencia de salida
        target = [num_dic[n] for n in (seq[1] + 'E')]      # Añadir 'E' al final de la secuencia objetivo

        # One-hot encoding de entrada y salida
        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target)  # El objetivo es una lista de índices (no one-hot)

    # Convertir listas a tensores de PyTorch
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Función para crear lote de prueba a partir de una palabra de entrada
def make_testbatch(input_word):
    input_w = input_word + 'P' * (n_step - len(input_word))  # Rellenar si es necesario
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]    # Salida inicializada con 'S' seguido de 'P'

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Definición del modelo Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        # RNN para codificación
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # RNN para decodificación
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        # Capa totalmente conectada para predecir clases
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        # Cambiar orden de dimensiones para las RNNs
        enc_input = enc_input.transpose(0, 1)  # [n_step, batch_size, n_class]
        dec_input = dec_input.transpose(0, 1)  # [n_step, batch_size, n_class]

        _, enc_states = self.enc_cell(enc_input, enc_hidden)   # Propagación en codificador
        outputs, _ = self.dec_cell(dec_input, enc_states)      # Propagación en decodificador

        modelo = self.fc(outputs)  # Aplicar capa final de predicción
        return modelo

# Inicializar modelo, función de pérdida y optimizador
modelo = Seq2Seq()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelo.parameters(), lr=0.001)

# Preparar los lotes
input_batch, output_batch, target_batch = make_batch()

# Entrenamiento del modelo
for epoch in range(5000):
    hidden = torch.zeros(2, batch_size, n_hidden)  # Estado oculto inicial (2 capas)

    optimizer.zero_grad()
    output = modelo(input_batch, hidden, output_batch)
    output = output.transpose(0, 1)  # [batch_size, n_step+1, n_class]

    # Cálculo de la pérdida acumulada sobre el lote
    loss = 0
    for i in range(batch_size):
        loss += criterion(output[i], target_batch[i])

    if (epoch + 1) % 1000 == 0:  # Mostrar el costo cada 1000 épocas
        print(f'Época: {epoch + 1:04d}, costo = {loss.item():.6f}')

    loss.backward()
    optimizer.step()

# Estrategias de decodificación

# Decodificación greedy (elige siempre el token de mayor probabilidad)
def greedy_decode(input_batch, hidden, output_batch):
    output = modelo(input_batch, hidden, output_batch)
    predict = output.data.max(2, keepdim=True)[1]
    return predict

# Decodificación usando Beam Search
def beam_search_decode(input_batch, hidden, output_batch, beam_width=3):
    output = modelo(input_batch, hidden, output_batch)
    output = F.softmax(output, dim=2)  # Convertir logits a probabilidades

    sequences = [[list(), 1.0]]  # Inicializar con secuencia vacía y score 1.0
    for row in output.squeeze(1):
        all_candidates = []
        for seq, score in sequences:
            for j in range(len(row)):
                candidate = [seq + [j], score * row[j].item()]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
        sequences = ordered[:beam_width]  # Mantener las mejores beam_width secuencias

    best_sequence = sequences[0][0]
    return torch.tensor(best_sequence, dtype=torch.long).view(1, -1, 1)

# Muestreo aleatorio con temperatura
def random_sample_with_temperature(output, temperature=1.0):
    output = output.div(temperature).exp()
    probs = F.softmax(output, dim=-1)
    return torch.multinomial(probs.view(-1, probs.size(-1)), 1).view(output.size(0), output.size(1), -1)

# Decodificación usando muestreo con temperatura
def decode_with_temperature(input_batch, hidden, output_batch, temperature=1.0):
    output = modelo(input_batch, hidden, output_batch)
    sampled_output = random_sample_with_temperature(output, temperature)
    return sampled_output

# Función general para traducir una palabra usando diferentes estrategias
def translate(word, strategy='greedy', beam_width=3, temperature=1.0):
    input_batch, output_batch = make_testbatch(word)
    hidden = torch.zeros(2, 1, n_hidden)

    if strategy == 'greedy':
        predict = greedy_decode(input_batch, hidden, output_batch)
    elif strategy == 'beam_search':
        predict = beam_search_decode(input_batch, hidden, output_batch, beam_width)
    elif strategy == 'temperature':
        predict = decode_with_temperature(input_batch, hidden, output_batch, temperature)

    decoded = [char_arr[i] for i in predict.squeeze()]
    if 'E' in decoded:  # Cortar la secuencia si se encuentra el final 'E'
        end = decoded.index('E')
        translated = ''.join(decoded[:end])
    else:
        translated = ''.join(decoded)

    return translated.replace('P', '')  # Eliminar caracteres de relleno

# Comparativa entre estrategias de decodificación
words = ['man', 'mans', 'king', 'black', 'upp']

print('Comparativa de decodificación:\n')
for word in words:
    print(f'Palabra: {word}')
    print(f'Greedy: {translate(word, strategy="greedy")}')
    print(f'Beam Search: {translate(word, strategy="beam_search", beam_width=3)}')
    print(f'Temperatura (T=1.0): {translate(word, strategy="temperature", temperature=1.0)}')
    print('---------------------------------')


### **Ejercicios**

1 .Implementa un mecanismo de atención en el modelo Seq2Seq para mejorar la traducción.

Instrucciones:

- Añade una capa de atención al modelo Seq2Seq.
- Modifica el método forward para incorporar la atención.
- Entrena el modelo y compara el rendimiento con el modelo original.

Pistas:

- Utiliza la clase nn.Linear para calcular los pesos de atención.
- Multiplica los pesos de atención con los estados ocultos del codificador para obtener el contexto.
- Concatena el contexto con la entrada del decodificador en cada paso de tiempo.

2 . Compara el rendimiento de las estrategias de decodificación Greedy, Beam Search y Temperatura en diferentes configuraciones de entrenamiento.

Instrucciones:

- Entrena el modelo con diferentes tamaños de conjunto de datos y configuraciones de hiperparámetros (por ejemplo, diferentes tamaños de hidden_size y num_layers).
- Aplica las tres estrategias de decodificación a cada modelo entrenado.
- Evalúa y compara la calidad de las traducciones utilizando métricas como la precisión y el BLEU score.

Pistas:

- Usa conjuntos de datos grandes y pequeños para ver cómo cambia el rendimiento.
- Experimenta con diferentes beam_width y temperatures.

3 . Implementa una variante de Beam Search que penalice secuencias más largas para evitar repeticiones innecesarias.

Instrucciones:

- Modifica la función beam_search_decode para incluir una penalización de longitud.
- Ajusta el cálculo de las puntuaciones de las secuencias para penalizar las secuencias más largas.
- Compara los resultados con el Beam Search estándar.

Pistas:

- Multiplica la puntuación de cada secuencia por una función de penalización basada en su longitud.
- Puedes usar una función de penalización lineal o exponencial.

4 . Implementa una estrategia de decodificación por temperatura que ajuste dinámicamente la temperatura en cada paso de tiempo.

Instrucciones:

- Modifica la función decode_with_temperature para ajustar la temperatura en cada paso de tiempo.
- Implementa una función que disminuya la temperatura a medida que avanza la decodificación, incentivando exploración al principio y explotación al final.
- Evalúa el impacto de la temperatura dinámica en la calidad de las traducciones.

Pistas:

- Usa una función de decremento lineal o exponencial para la temperatura.
- Compara los resultados con una temperatura fija.

5 . Analiza la complejidad computacional y el rendimiento de las diferentes estrategias de decodificación.

Instrucciones:

- Mide el tiempo de ejecución de las estrategias Greedy, Beam Search y Temperatura para diferentes tamaños de vocabulario y longitudes de secuencia.
- Analiza cómo cambia la complejidad computacional con respecto a beam_width y temperature.
- Discute los trade-offs entre la calidad de la traducción y el tiempo de ejecución.

Pistas:

- Usa la biblioteca time para medir el tiempo de ejecución.
- Realiza pruebas con diferentes configuraciones y grafica los resultados.


6 .Mejora la generalización del modelo incorporando técnicas de regularización y dropout.

Instrucciones:

- Añade capas de dropout adicionales al modelo Seq2Seq.
- Implementa técnicas de regularización como L2.
- Entrena el modelo con estas técnicas y compara el rendimiento con el modelo original.

Pistas:

- Usa nn.Dropout en las capas RNN y totalmente conectadas.
- Ajusta los hiperparámetros de regularización y dropout para encontrar la configuración óptima

In [ ]:
## Tus respuestas